In [ ]:
import logging

import numpy as np

from atmodeller import (
    ChemicalSpecies,
    EquilibriumModel,
    Planet,
    SpeciesNetwork,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# Basic Usage

This notebook is available at `notebooks/basic_usage.ipynb` and is easiest to obtain by downloading the source code.

## Species and thermodynamic data

The species available in *Atmodeller* can be found in the `thermodata` subpackage, where the prefix of the dictionary key denotes the chemical formula in *Hill notation* and the suffix describes the *states of aggregation* in accordance with the JANAF convention.

In [ ]:
# Get all available species
available_species = SpeciesNetwork.available_species()
logger.info("Available species = %s", available_species)

# To create a gas species, for example CO2, where the state of aggregation defaults to 'g':
CO2_g = ChemicalSpecies.create_gas("CO2")

# The unique name that Atmodeller assigns combines the Hill notation and the state of aggregation
logger.info("Species name = %s", CO2_g.data.name)

# Compute the Gibbs energy relative to RT at 2000 K
temperature = 2000.0
gibbs = CO2_g.data.get_gibbs_over_RT(temperature)
logger.info("Gibbs/RT = %s", gibbs)

# Compute the composition
composition = CO2_g.data.composition
logger.info("Composition = %s", composition)

# Access more thermodynamic data
heat_capacity = CO2_g.data.thermo.cp(temperature)
logger.info("Heat capacity = %s", heat_capacity)

# Etc., other methods are available to compute other quantities

## Solubility

Solubility laws are available in the `solubility` subpackage.

In [ ]:
solubility_models = get_solubility_models()
logger.info("Solubility models = %s", solubility_models.keys())

CO2_basalt = solubility_models["CO2_basalt_dixon95"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = CO2_basalt.concentration(0.5, temperature=1300, pressure=1)
logger.info("Concentration (ppmw) = %s", concentration)

In [ ]:
N2_basalt = solubility_models["N2_basalt_libourel03"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt.concentration(0.20, temperature=1698.15, pressure=1, fO2=10**-16.2)
logger.info("Concentration (ppmw) = %s", concentration)

In [ ]:
N2_basalt_dasgupta = solubility_models["N2_basalt_dasgupta22"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt_dasgupta.concentration(
    1550, temperature=1773.15, pressure=1708.7, fO2=1.8e-13
)
logger.info("Concentration (ppmw) = %s", concentration)

## Real gas EOS

Real gas equations of state are available in the `eos` subpackage.

In [ ]:
# Get all available EOS models
eos_models = get_eos_models()
logger.info("EOS models = %s", eos_models.keys())

# Get a CH4 model
CH4_eos_model = eos_models["CH4_beattie_holley58"]
# Compute the fugacity at 800 K and 100 bar
fugacity = CH4_eos_model.fugacity(800, 100)
logger.info("Fugacity = %s bar", fugacity)
# Compute the compressibility factor at the same conditions
compressibility = CH4_eos_model.compressibility_factor(800, 100)
logger.info("Compressibility factor = %s", compressibility)
# Etc., other methods are available to compute other quantities

We can also use broadcasting to perform multiple evaluations at once, for example to compute a grid of fugacities:

In [ ]:
# Define the temperature (K) and pressure (bar) grid
temperature = np.array([1000, 1600])
pressure = np.array([1, 10, 100])

temperature_broadcasted = temperature[:, None]
pressure_broadcasted = pressure[None, :]

# Get a CH4 model
CH4_eos_model = eos_models["CH4_cork_cs_holland91"]
# Compute the fugacity
fugacity = CH4_eos_model.fugacity(temperature_broadcasted, pressure_broadcasted)
logger.info("Fugacity = %s bar", fugacity)
# Compute the compressibility factor at the same conditions
compressibility = CH4_eos_model.compressibility_factor(
    temperature_broadcasted, pressure_broadcasted
)
logger.info("Compressibility factor = %s", compressibility)
# Etc., other methods are available to compute other quantities

## Model with mass constraints

A common scenario is to calculate how volatiles partition between a magma ocean and an atmosphere when the total elemental abundances are constrained. `Planet()` defaults to a molten Earth, but the planetary parameters can be changed using input arguments.

In [ ]:
solubility_models = get_solubility_models()

H2_g = ChemicalSpecies.create_gas("H2")
H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = ChemicalSpecies.create_gas("O2")

# This is one way of defining the species collection, and this approach is preferred if you want to
# specify species with solubility laws, as defined for H2O_g above.
species = SpeciesNetwork((H2_g, H2O_g, O2_g))

# Planet has input arguments that you can change. See the class documentation.
planet = Planet()
model = EquilibriumModel(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {"H": h_kg, "O": o_kg}

model.solve(state=planet, mass_constraints=mass_constraints)
output = model.output

# Quick look at the solution
solution = output.quick_look()
logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Get the complete solution as dataframes
# solution_dataframes = output.to_dataframes()

# Write the complete solution to Excel
# output.to_excel("example_mass_constraints")

## Model with fO2 constraint

Another common scenario is to calculate how volatiles partition between a magma ocean and an atmosphere when fO2 is fixed relative to a buffer and the total elemental abundances are constrained.

In [ ]:
H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
H2_g = ChemicalSpecies.create_gas("H2")
O2_g = ChemicalSpecies.create_gas("O2")

species = SpeciesNetwork((H2O_g, H2_g, O2_g))

planet = Planet()
model = EquilibriumModel(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
mass_constraints = {"H": h_kg}

# Use the Iron Wustite buffer.  The "-1" argument is the log10 shift relative to the buffer.
fugacity_constraints = {"O2_g": IronWustiteBuffer(-1)}

model.solve(
    state=planet, fugacity_constraints=fugacity_constraints, mass_constraints=mass_constraints
)
output = model.output

# Quick look at the solution
solution = output.quick_look()
logger.info("solution = %s", solution)

## Defining a network of species

In the above example, a species network was created by first creating the species and then aggregating the species into a `SpeciesNetwork`. This approach allows for complete generality since each species can also have its own solubility law assigned. However, if you want to create a simple gas network that assumes ideality and no solubility, you can also use the following formulation, where the *state of aggregation* must be specified after the formula (and separated by an underscore) for all species:

In [ ]:
species_no_solubility = SpeciesNetwork.create(("H2_g", "H2O_g", "O2_g"))
logger.info("species_no_solubility = %s", species_no_solubility)

## Batch calculation

For a batch calculation you can provide arrays to the planet or constraints. All arrays must have the same size because for a batch calculation the array values are aligned by position. Single values will automatically be broadcasted to the maximum array size.

In [ ]:
solubility_models = get_solubility_models()

H2_g = ChemicalSpecies.create_gas("H2")
H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = ChemicalSpecies.create_gas("O2")

species = SpeciesNetwork((H2_g, H2O_g, O2_g))

# Batch temperature and radius, where the entries correspond by position. You could also choose
# to leave one or both as scalars.
surface_temperature = np.array([2000, 2000, 1500, 1500])
earth_radius = 6371000  # m
surface_radius = earth_radius * np.array([1.5, 3, 1.5, 3])

planet = Planet(temperature=surface_temperature, surface_radius=surface_radius)
model = EquilibriumModel(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
scale_factor = 5
mass_constraints = {
    # We can also batch constraints, as long as we also have a total of 4 entries
    "H": np.array([h_kg, h_kg, h_kg * scale_factor, h_kg * scale_factor]),
    "O": np.array([o_kg, o_kg * scale_factor, o_kg, o_kg * scale_factor]),
}

# Initial solution guess number of moles
initial_log_number_moles = 50

model.solve(
    state=planet,
    initial_log_number_moles=initial_log_number_moles,
    mass_constraints=mass_constraints,
)
output = model.output

# Quick look at the solution
solution = output.quick_look()
logger.info("Quick look = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_batch")

## Monte Carlo

Exploring atmospheric compositions in a Monte Carlo model can be achieved with a batch 
calculation over a range of parameters. Note that in this case the same initial solution is used 
for all cases.

In [ ]:
solubility_models = get_solubility_models()

H2_g = ChemicalSpecies.create_gas("H2")
H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = ChemicalSpecies.create_gas("O2")

species = SpeciesNetwork((H2_g, H2O_g, O2_g))
planet = Planet()
model = EquilibriumModel(species)

number_of_realisations = 1000
log10_number_oceans = np.random.uniform(0, 3, number_of_realisations)
number_oceans = 10**log10_number_oceans
fO2_min = -3
fO2_max = 3
fO2_log10_shifts = np.random.uniform(fO2_min, fO2_max, number_of_realisations)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(number_oceans)
mass_constraints = {"H": h_kg}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number of moles
initial_log_number_moles = 50 * np.ones(len(species))

model.solve(
    state=planet,
    initial_log_number_moles=initial_log_number_moles,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output = model.output

# Quick look at the solution
# solution = output.quick_look()
# logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_monte_carlo")